In [1]:
// Import max-plus algebra functions from our library
import { 
  oplus, otimes, odot, transpose, trace, star, plus, powOtimes,
  EPS, INF, Matrix, Vector,
  createEpsilonMatrix, createIdentityMatrix, reset as resetMatrix
} from './max-plus';

console.log("Max-plus algebra library imported successfully");


Max-plus algebra library imported successfully


### Delay

<img src="../../img/Delay.svg" alt="drawing" width="500"/>

In [2]:
// System Max-Plus Encoding

// Create matrices for system
let x: Matrix = [[EPS, EPS]];

// Indices for system
const a = 0;
const b = 1;

console.log("System setup complete");


System setup complete


In [3]:
// Logical time
let t: number = EPS;
let tBar: Matrix = [[t, t]];

// Timer
const offset = 0;
const period = 1;
let timerValue: number = offset;

// Delay
const delay = 0.1;

function updateLogicalTime(k: number): void {
  if (k == 0) t = timerValue;
  // If k is odd, add delay on top of t.
  else if (k % 2 == 1) t += delay;
  // If k is even, increment timerValue.
  else {
    timerValue += period;
    t = timerValue; 
  }
  // Update tBar.
  tBar = [[t, t]];
}

// Execution times
let e: Matrix = [[0.1, 0.1]];

function updateApparentLatency(k: number): void {
  // If k is even, both reactions fire.
  if (k % 2 == 0) e = [[0.1, 0.1]];
  // If k is odd, only reaction A fires.
  else [[0.1, 0]];
}

In [4]:
// Evolution equations for system in factored form: x' = (GammaStar ⊗ B) ⊗ x ⊕ (GammaStar ⊗ tBar)
const Gamma: Matrix = [
  [0, EPS],
  [e[0][a], 0],
];
const GammaStar: Matrix = otimes(Gamma, Gamma);

// B is diagonal with execution times on the diagonal
const B: Matrix = [
  [e[0][a], e[0][b]],
  [e[0][a], e[0][b]],
];

console.log("Gamma:", Gamma);
console.log("GammaStar:", GammaStar);
console.log("B (diagonal):", B);


Gamma: [ [ 0, -Infinity ], [ 0.1, 0 ] ]
GammaStar: [ [ 0, -Infinity ], [ 0.1, 0 ] ]
B (diagonal): [ [ 0.1, 0.1 ], [ 0.1, 0.1 ] ]


In [5]:
// Step function for system evolution using GammaStar and B
function step(x: Matrix, tBar: Matrix): Matrix {
  const xT: Matrix = transpose(x);
  const tT: Matrix = transpose(tBar);

  // (GammaStar ⊗ B) ⊗ x
  const GB = otimes(GammaStar, B);
  const GBx = otimes(GB, xT);

  // GammaStar ⊗ tBar
  const Gt = otimes(GammaStar, tT);

  // Combine and transpose back to row vector
  const result = oplus(GBx, Gt);
  return transpose(result);
}

// Reporting function (updated to work with matrices)
function reportStats(k: number, x: Matrix, t: number): void {
  const lagK = x[0].map(xi => xi - t); // lag

  console.log(`Tag index k=${k}`);
  console.log(`t(k)= ${t}`);
  console.log(`x(k) = earliest possible firing times = [${x[0].join(', ')}]`);
  console.log(`lag(k) = [${lagK.join(', ')}]`);
}  

console.log("System functions defined");


System functions defined


In [6]:
// System simulation setup
// Initial logical time (scalar then blasted)
t = 0;
tBar = [[t, t]];

// Reset x to eps using the library function
x = resetMatrix(x);

// Iteration count
let k = 0;

console.log("System simulation ready");


System simulation ready


### Execute the following repeatly.

In [11]:
// Update logical time and apparent latency.
updateLogicalTime(k);
updateApparentLatency(k);

// Step the system from k to k+1
x = step(x, tBar);

// Statistics
reportStats(k, x, t);

// Bump k
k += 1;


Tag index k=4
t(k)= 2
x(k) = earliest possible firing times = [2, 2.1]
lag(k) = [0, 0.10000000000000009]
5
